In [70]:
import numpy as np
import pandas as pd
import re
import cupy as cp
from tqdm import tqdm
import spacy
from spacy.tokens import Doc
from functools import partial
import sys
sys.path.append('c:\\python312\\lib\\site-packages')
# Load the data
df = pd.read_csv("data/bbc-text.csv")
def create_vocabulary(complete_text):
    text = re.sub(r'[^\w\s]', ' ', complete_text).split()
    vocabulary = list(set(text)) 
    vocabulary = {word: idx for idx, word in enumerate(vocabulary)} 
    print("Vocabulary size: ", len(vocabulary))
    return vocabulary

complete_text = ' '.join(df["text"].tolist()) 
vocabulary= create_vocabulary(complete_text)
vocabulary

Vocabulary size:  29457


{'intertwined': 0,
 'perpetuating': 1,
 'dolls': 2,
 'compton': 3,
 'purporting': 4,
 'ilw': 5,
 'rice': 6,
 'feted': 7,
 'mccurry': 8,
 'qualification': 9,
 'restructuring': 10,
 'redesigned': 11,
 'viktor': 12,
 '56bn': 13,
 'elene': 14,
 'pointed': 15,
 '15m': 16,
 'fork': 17,
 'treasure': 18,
 'icstis': 19,
 'united': 20,
 'walshe': 21,
 'tribunal': 22,
 'chesney': 23,
 'creations': 24,
 'freak': 25,
 'achilles': 26,
 'tutsis': 27,
 'landing': 28,
 'woman': 29,
 'consulted': 30,
 'maximise': 31,
 'malone': 32,
 'apocalypse': 33,
 'commoditising': 34,
 'bouira': 35,
 'musicians': 36,
 'cockerill': 37,
 'las': 38,
 'speechless': 39,
 'ferdinand': 40,
 '110m': 41,
 '76m': 42,
 'fenton': 43,
 'husbands': 44,
 'rugged': 45,
 'barclay': 46,
 'handbook': 47,
 'rare': 48,
 'chains': 49,
 'dame': 50,
 'noting': 51,
 'basf': 52,
 'honor': 53,
 'spectacularly': 54,
 'preston': 55,
 'alzheimer': 56,
 'foment': 57,
 'dying': 58,
 'sectors': 59,
 'emerged': 60,
 'server': 61,
 'offer': 62,
 'duf

In [71]:
def input_one_hot_vectors(words,vocabulary):
    
    inputs=np.zeros((len(words),len(vocabulary)),int)
     
    for i in range(len(words)): 
        inputs[i][vocabulary[words[i]]]=1
    
    return inputs 

word=cp.array(input_one_hot_vectors(['intertwined'],vocabulary))
word.shape,words_len_embedding_layer.shape,words_len_embedding_bias.shape

((1, 29457), (29457, 100), (100,))

In [74]:
import pickle
with open('data/words_len_embedding_layer.pkl', 'rb') as f:
    words_len_embedding_layer = pickle.load(f)

with open('data/words_len_embedding_bias.pkl', 'rb') as f:
    words_len_embedding_bias = pickle.load(f)

# with open('data/outlayer_maps_vocab_average.pkl', 'rb') as f:
#     outlayer_maps_vocab_average = pickle.load(f)

# with open('data/out_bias_maps_vocab_average.pkl', 'rb') as f:
#     out_bias_maps_vocab_average = pickle.load(f)

# with open('data/outlayer_maps_vocab_concat.pkl', 'rb') as f:
#     outlayer_maps_vocab_concat = pickle.load(f)

# with open('data/out_bias_maps_vocab_concat.pkl', 'rb') as f:
#     out_bias_maps_vocab_concat = pickle.load(f)

In [75]:
import numpy as np
import cupy as cp

words = []  # List to store words
embeddings_list = []  # List to store word embeddings

# Loop over your vocabulary
for word in vocabulary.keys():
    words.append(word)  # Append word to the list
    
    # Create one-hot vector for the word using CuPy
    word_one_hot = cp.array(input_one_hot_vectors([word], vocabulary))
    
    # Compute the word vector using matrix multiplication (assuming `words_len_embedding_layer` and `words_len_embedding_bias` are CuPy arrays)
    word_vector = word_one_hot @ words_len_embedding_layer + words_len_embedding_bias
    
    # Append the computed word vector to the list
    embeddings_list.append(word_vector.get())  # Convert to NumPy array before appending

# After the loop, convert the list of embeddings to a NumPy array
words_embeddings=np.array([x[0]  for x in embeddings_list] ) 

# Save the embeddings to a TSV file
np.savetxt('data/learned_embeddings.tsv', words_embeddings, delimiter='\t')

# Save the words to a metadata TSV file
with open('data/metadata.tsv', 'w') as f:
    for word in words:
        f.write(f"{word}\n")
